In [1]:
from models.rnn.rnn_plain import RNN_Plain
from models.rnn.rnn_attention import RNN_Attention

# from lang_numbers import load_data_int_seq
from lang_numbers import load_data_int_seq
from utils import accuracy_score

%load_ext autoreload

In [2]:
%autoreload 2

In [4]:
output_lang, input_lang, train, X_test, y_test = load_data_int_seq()

[[ 4  8 12 16 20 24 28 32]]
-
0
1
2
3
4
5
6
7
8
9
t
+
3
*
t
-
1
+
1
7
+
4
*
9
-
1
-
t


## Plain RNN

In [5]:
algo = RNN_Plain(symbols = "+*-0123456789t", 
output_sequence_length = 9, 
encoded_seq_length = 9, 
num_epochs = 1, 
input_size = input_lang.n_words, 
hidden_size = 256, 
output_size=output_lang.n_words, 
embedding_size = 28, 
batch_size = 1, 
num_gru_layers = 2,
dropout_prob = 0.3,
calc_magnitude_on=False)


EncoderRNN(
  (embedding): Embedding(13, 28)
  (gru): GRU(28, 256, num_layers=2, dropout=0.3)
)
DecoderRNN(
  (embedding): Embedding(11, 28)
  (gru): GRU(28, 256, num_layers=2, dropout=0.3)
  (out): Linear(in_features=256, out_features=11, bias=True)
  (softmax): LogSoftmax(dim=1)
)


In [11]:
algo.train(input_lang, output_lang, train)


['4,8,12,16,20,24,28,32']
test_char  tensor([[7],
        [0]], device='cuda:0')
[[tensor([[7],
        [0]], device='cuda:0')], [tensor([[11],
        [ 0]], device='cuda:0')], [tensor([[4],
        [0]], device='cuda:0'), tensor([[5],
        [0]], device='cuda:0')]]
[tensor([[7],
        [0]], device='cuda:0')]


RuntimeError: Tensor.__contains__ only supports Tensor or scalar, but you passed in a <class 'str'>.

In [ ]:
pred = algo.infer(input_lang, output_lang, X_test[:1000])
pred[:10]

In [ ]:
accuracy_score(pred, y_test[:1000])

### Plain RNN with Magnitude loss

In [ ]:
magn_algo = RNN_Plain(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 5000, input_size=input_lang.n_words, hidden_size=512, output_size=output_lang.n_words, calc_magnitude_on=True)
magn_algo.train(input_lang, output_lang, train)

In [ ]:
pred = magn_algo.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

In [ ]:
accuracy_score(pred, y_test[:1000])

##  Attention-based

In [ ]:
algo_attn = RNN_Attention(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 5000, input_size=input_lang.n_words, hidden_size=512, output_size=output_lang.n_words, calc_magnitude_on=False)
algo_attn.train(input_lang, output_lang, train)

In [ ]:
pred = algo_attn.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

In [ ]:
accuracy_score(pred, y_test[:1000])

### Attention with magnitude loss

In [ ]:
algo_attn_magn = RNN_Attention(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 5000, input_size=input_lang.n_words, hidden_size=512, output_size=output_lang.n_words, calc_magnitude_on=True)
algo_attn_magn.train(input_lang, output_lang, train)

In [ ]:
pred = algo_attn_magn.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

In [ ]:
accuracy_score(pred, y_test[:1000])

# Helper Stuff

In [ ]:
import torch
import torch.nn as nn
m = nn.LogSoftmax()
input = torch.randn(2, 3)
print(input)
output = m(input)
print(output)

In [ ]:
# 2D loss example (used, for example, with image inputs)
N, C = 5, 4
loss = nn.NLLLoss()

# input is of size N x C x height x width
data = torch.randn(N, 16, 10, 10)
conv = nn.Conv2d(16, C, (3, 3))
m = nn.LogSoftmax(dim=1)

# print(data.shape)
activated = m(conv(data))
print("output: ", activated.shape)

# each element in target has to have 0 <= value < C
target = torch.empty(N, 8, 8, dtype=torch.long).random_(0, C)
print("target: ", target.shape)
output = loss(activated, target)

# output.backward()

In [ ]:
m = nn.LogSoftmax(dim=1)
loss = nn.NLLLoss()
# input is of size N x C = 3 x 5
# input = minibatchsize, number of categories

input = torch.randn(3, 5, requires_grad=True)
# print(input.shape)
activation = m(input)
print(activation.shape)
# each element in target has to have 0 <= value < C
target = torch.tensor([1, 0, 4])
print(target.shape)
output = loss(activation, target)
# output.backward()

In [ ]:
from line_profiler import LineProfiler

stuff_to_evaluate = [1,2,3]
def function_to_evaluate(stuff_to_evaluate):
    pass

lp = LineProfiler()
lp_wrapper = lp(function_to_evaluate)
lp_wrapper(stuff_to_evaluate)
lp.print_stats()